In [ ]:
import threading
import time
import queue

task_queue = queue.SimpleQueue()
def producer():
    """生产者线程"""
    try:
        for i in range(5):
            item = f"任务{i}"
            task_queue.put(item)
            print(f"[生产者] 添加了任务: {item}")
            time.sleep(0.5)  # 模拟耗时操作
    except queue.Full:
        print("[生产者] 队列已满，无法添加任务。")

def consumer():
    """消费者线程"""
    while True:
        try:
            item = task_queue.get(timeout=1)  # 尝试获取，但此时可能已空
            if item is None:
                print("[消费者] 遇到退出标识，退出。")
                break
            print(f"[消费者] 处理任务: {item}")
            time.sleep(0.5)
        except queue.Empty:
            print("[消费者] 一段时间内没有新任务，退出。")

prod_thread = threading.Thread(target=producer)
cons_thread = threading.Thread(target=consumer)

prod_thread.start()
cons_thread.start()

# 等待生产者添加完所有任务
prod_thread.join()

#添加退出标识
task_queue.put(None)

# 等待消费者处理完所有任务
cons_thread.join()
print("程序结束。")


[生产者] 添加了任务: 任务0
[消费者] 处理任务: 任务0
[生产者] 添加了任务: 任务1
[消费者] 处理任务: 任务1
[生产者] 添加了任务: 任务2[消费者] 处理任务: 任务2

[生产者] 添加了任务: 任务3[消费者] 处理任务: 任务3

[生产者] 添加了任务: 任务4[消费者] 处理任务: 任务4

[消费者] 遇到退出标识，退出。
程序结束。


# 动态任务

In [ ]:
import time
import queue
from enum import Enum
from dataclasses import dataclass
from typing import Callable, Any

# 定义任务状态枚举
class TaskStatus(Enum):
    PENDING = "等待中"
    RUNNING = "运行中"
    PAUSED = "已暂停"
    CANCELLED = "已取消"
    FINISHED = "已完成"

# 使用数据类来定义任务，更清晰
@dataclass
class Task:
    id: int|str
    name: str
    action: Callable[[], Any]  # 任务要执行的无参数函数
    status: TaskStatus = TaskStatus.PENDING


class TaskQueueManager:
    def __init__(self):
        # 任务队列，存放待执行的任务
        self.task_queue = queue.Queue()
        # 字典，用于通过任务ID快速查找和操作任务
        self.all_tasks = {}
        # 控制工作线程运行的标志
        self._running = False
        # 工作线程对象
        self._worker_thread = None
        # 线程锁，确保对任务状态的修改是线程安全的
        self._lock = threading.Lock()

    def add_task(self, name: str, action: Callable[[], Any]):
        """添加一个新任务到队列"""
        with self._lock:
            # 生成一个简单的任务ID（实际应用中可能需要更健壮的方法）
            task_id = f"{name}_{len(self.all_tasks)}"
            new_task = Task(id=task_id, name=name,action=action)
            self.all_tasks[task_id] = new_task

            # 只有非暂停状态的任务才放入执行队列
            if new_task.status == TaskStatus.PENDING:
                self.task_queue.put(new_task)
            print(f"任务已添加: {name} (ID: {task_id})")
            return task_id
     
    def pause_task(self, task_id):
        """暂停一个任务（如果它在等待中）"""
        with self._lock:
            task = self.all_tasks.get(task_id)
            if task and task.status == TaskStatus.PENDING:
                task.status = TaskStatus.PAUSED
                print(f"任务已暂停: {task.name} (ID: {task_id})")
            # 注意：对于正在运行的任务，简单的暂停需要更复杂的机制（如线程信号）
        
    def resume_task(self, task_id):
        """恢复一个被暂停的任务"""
        with self._lock:
            task = self.all_tasks.get(task_id)
            if task and task.status == TaskStatus.PAUSED:
                task.status = TaskStatus.PENDING
                # 恢复后，将任务重新放入执行队列
                self.task_queue.put(task)
                print(f"任务已恢复: {task.name} (ID: {task_id})")

    def delete_task(self, task_id):
        """删除一个任务（如果可能）"""
        with self._lock:
            task = self.all_tasks.get(task_id)
            if task:
                if task.status == TaskStatus.RUNNING:
                    print(f"警告：无法直接删除正在运行的任务 {task.name}。可以尝试取消。")
                    # 对于运行中的任务，通常需要设置取消标志，在任务函数内部检查
                elif task.status in (TaskStatus.PENDING, TaskStatus.PAUSED):
                    task.status = TaskStatus.CANCELLED
                    # 从 all_tasks 字典中移除
                    self.all_tasks.pop(task_id, None)
                    print(f"任务已删除: {task.name} (ID: {task_id})")
                elif task.status == TaskStatus.CANCELLED:
                    print(f"任务早已被删除。")
            else:
                print(f"未找到ID为 {task_id} 的任务。")

    def _worker(self):
        """工作线程的主循环函数"""
        while self._running or not self.task_queue.empty():
            try:
                # 从队列中获取任务，设置超时以便能响应停止信号
                task = self.task_queue.get(timeout=0.1)
            except queue.Empty:
                continue  # 如果队列为空，继续循环

            with self._lock:
                # 再次检查任务状态，因为它可能在出队后被暂停或取消
                if task.status != TaskStatus.PENDING:
                    self.task_queue.task_done()
                    continue
                task.status = TaskStatus.RUNNING

            print(f"[执行开始] {task.name}")
            try:
                # 执行任务函数
                result = task.action()
                with self._lock:
                    task.status = TaskStatus.FINISHED
                print(f"[执行成功] {task.name} -> 结果: {result}")
            except Exception as e:
                print(f"[执行失败] {task.name} -> 错误: {e}")
                with self._lock:
                    task.status = TaskStatus.CANCELLED  # 或设置为 FAILED
            finally:
                # 告知队列该任务已处理完
                self.task_queue.task_done()

    def start(self):
        """启动任务队列管理器"""
        if not self._running:
            self._running = True
            self._worker_thread = threading.Thread(target=self._worker, daemon=True)
            self._worker_thread.start()
            print("任务队列管理器已启动。")

    def stop(self):
        """停止任务队列管理器，等待所有剩余任务完成"""
        print("正在停止任务队列管理器...")
        self._running = False
        if self._worker_thread:
            self._worker_thread.join(timeout=5)  # 等待工作线程结束，最多等5秒
        print("任务队列管理器已停止。")

    def wait_until_all_done(self):
        """阻塞主线程，直到所有任务执行完毕（包括队列中和正在运行的）"""
        self.task_queue.join()  # 等待队列中所有任务被标记为 done
        print("所有任务均已处理完毕。")


# 示例任务函数
def sample_task_1():
    time.sleep(2)
    return "任务1完成"

def sample_task_2():
    time.sleep(1)
    return "任务2完成"

def long_running_task():
    for i in range(5):
        print(f"  长时间任务进行中... 步骤 {i+1}/5")
        time.sleep(1)
    return "长时间任务完成"

# 演示用法
manager = TaskQueueManager()
manager.start()

# 添加几个任务
task1_id = manager.add_task("快速任务1", sample_task_1)
task2_id = manager.add_task("快速任务2", sample_task_2)
long_task_id = manager.add_task("长时间后台任务", long_running_task)

# 演示暂停和恢复
manager.pause_task(task2_id)
time.sleep(1)
manager.resume_task(task2_id)

# 演示删除（取消)一个等待中的任务
time.sleep(0.5) # 确保任务已添加但未开始
manager.delete_task(task1_id)

# 等待所有任务完成
try:
    manager.wait_until_all_done()
    time.sleep(1)  # 给日志输出一点时间
except KeyboardInterrupt:
    print("\n接收到中断信号...")
finally:
    manager.stop()


任务队列管理器已启动。
任务已添加: 快速任务1 (ID: 快速任务1_0)
任务已添加: 快速任务2 (ID: 快速任务2_1)
任务已添加: 长时间后台任务 (ID: 长时间后台任务_2)
任务已暂停: 快速任务2 (ID: 快速任务2_1)
[执行开始] 快速任务1
任务已恢复: 快速任务2 (ID: 快速任务2_1)
警告：无法直接删除正在运行的任务 快速任务1。可以尝试取消。
[执行成功] 快速任务1 -> 结果: 任务1完成
[执行开始] 快速任务2
[执行成功] 快速任务2 -> 结果: 任务2完成
[执行开始] 长时间后台任务
  长时间任务进行中... 步骤 1/5
  长时间任务进行中... 步骤 2/5
  长时间任务进行中... 步骤 3/5
  长时间任务进行中... 步骤 4/5
  长时间任务进行中... 步骤 5/5
[执行成功] 长时间后台任务 -> 结果: 长时间任务完成
所有任务均已处理完毕。
正在停止任务队列管理器...
任务队列管理器已停止。
